In [ ]:
import arviz as az
import pandas as pd
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import warnings

from dataclasses import dataclass
from uplift_dgf import get_daily_visitors, get_daily_conversions, grid_configs
from typing import List
from tqdm.auto import tqdm

In [ ]:
warnings.filterwarnings("ignore", "Sampling:")

In [2]:
# Set plot options
plt.rcParams["figure.autolayout"] = True
plt.rcParams['figure.dpi'] = 100
sns.set_style("darkgrid")

In [3]:
# Set print options
np.set_printoptions(suppress=True, precision=4, edgeitems = 7)
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.max_columns', None)

In [4]:
# Create random generator
random_seed = 1923
rng = np.random.default_rng(random_seed)

## Set true parameters

In [5]:
# Avg. visitors per day & its noise, as Poisson distributions
lam_visitors = 100
lam_visitors_noise = 5

In [6]:
# Conversion rates
conversion_control = 0.15
conversion_experiment = 0.20
true_relative_uplift = (conversion_experiment /conversion_control) - 1

# Noise in conversion rates, as Beta distribution
# Mean = alpha / (alpha + beta)
conversion_noise_alpha = 3 
conversion_noise_beta = 100

In [ ]:
# Test conversion rate noise generation (will be randomly negated)
sns.displot(
    np.random.beta(conversion_noise_alpha, conversion_noise_beta, 1000)
)
_ = plt.title(f"Conversion rate noise distribution, Beta({conversion_noise_alpha}, {conversion_noise_beta})")

## Define data & model classes

In [8]:
@dataclass
class VariantData:
    visitors: int
    conversions: int

In [9]:
@dataclass
class ConversionPrior:
    alpha: float
    beta: float

In [10]:
class ConversionRateModel:

    def __init__(self, priors: List[ConversionPrior], data: List[VariantData], priors_label: str):

        # Set the conversion rate priors
        # 0: control, 1: experiment
        self.prior_control = priors[0]
        self.prior_experiment = priors[1]

        # Get experiment data
        # 0: control, 1: experiment
        self.visitors = [variant.visitors for variant in data]
        self.conversions = [variant.conversions for variant in data]

        self.priors_label = priors_label
        self.prior_predictive = None
        self.posterior_predictive = None
    
    def _create_model(self):

        # Define model
        with pm.Model() as model:

            # Conversion rate priors,
            conversion_rate_control = pm.Beta(
                "conversion_rate_control",
                alpha = self.prior_control.alpha,
                beta = self.prior_control.beta
            )

            conversion_rate_experiment = pm.Beta(
                "conversion_rate_experiment",
                alpha = self.prior_experiment.alpha,
                beta = self.prior_experiment.beta
            )

            # Likelihoods
            likelihood_control = pm.Binomial(
                "outcome_control",
                n = self.visitors[0],
                p = conversion_rate_control,
                observed = self.conversions[0]
            )

            likelihood_experiment = pm.Binomial(
                "outcome_experiment",
                n = self.visitors[1],
                p = conversion_rate_experiment,
                observed = self.conversions[1]
            )

            # Relative uplift
            relative_uplift = pm.Deterministic(
                "relative_uplift",
                (conversion_rate_experiment / conversion_rate_control) - 1
            )
            
        return model
    
    def get_prior_predictive(self, **kwargs):

        # Sample prior predictive distribution
        with self._create_model():
            self.prior_predictive = pm.sample_prior_predictive(**kwargs)
    
    def get_posterior_predictive(self, **kwargs):

        # Draw posterior samples
        with self._create_model():
            self.posterior_predictive = pm.sample(**kwargs)
    
    def plot_prior_predictive(self, kind = "kde", **kwargs):
        
        if self.prior_predictive == None:
            self.get_prior_predictive(**kwargs)
        
        priors_label = self.priors_label

        # Plot conversion rate priors
        fig, ax = plt.subplots(2, sharex = True)

        # Control
        _ = az.plot_posterior(
            self.prior_predictive.prior["conversion_rate_control"],
            kind = kind,
            ax = ax[0]
            )
        _ = ax[0].axvline(conversion_control, color = "red")
        _ = ax[0].annotate(str(round(conversion_control, 2)), (conversion_control, 0.2))
        _ = ax[0].set_title(f"Control conversion rate, {priors_label} prior predictive")

        # Experiment
        _ = az.plot_posterior(
            self.prior_predictive.prior["conversion_rate_experiment"],
            kind = kind,
            ax = ax[1]
            )
        _ = ax[1].axvline(conversion_experiment, color = "red")
        _ = ax[1].annotate(str(round(conversion_experiment, 2)), (conversion_experiment, 0.2))
        _ = ax[1].set_title(f"Experiment conversion rate, {priors_label} prior predictive")

        plt.show()
        plt.close()

        # Plot outcome prior
        _ = az.plot_posterior(
            self.prior_predictive.prior["relative_uplift"],
            kind = kind
            )
        _ = plt.axvline(true_relative_uplift, color = "red")
        _ = plt.annotate(str(round(true_relative_uplift, 2)), (true_relative_uplift, 0.1))
        _ = plt.title(f"Relative uplift, {priors_label} prior predictive")

        plt.show()
        plt.close()
    
    def plot_posterior_predictive(self, kind = "kde", **kwargs):
        
        if self.posterior_predictive == None:
            self.get_posterior_predictive(**kwargs)
        
        priors_label = self.priors_label

        # Plot conversion rate posteriors
        fig, ax = plt.subplots(2, sharex = True)

        # Control
        _ = az.plot_posterior(
            self.posterior_predictive.posterior["conversion_rate_control"],
            kind = kind,
            ax = ax[0]
            )
        _ = ax[0].axvline(conversion_control, color = "red")
        _ = ax[0].annotate(str(round(conversion_control, 2)), (conversion_control, 0.2))
        _ = ax[0].set_title(f"Control conversion rate, {priors_label} posterior predictive")

        # Experiment
        _ = az.plot_posterior(
            self.posterior_predictive.posterior["conversion_rate_experiment"],
            kind = kind,
            ax = ax[1]
            )
        _ = ax[1].axvline(conversion_experiment, color = "red")
        _ = ax[1].annotate(str(round(conversion_experiment, 2)), (conversion_experiment, 0.2))
        _ = ax[1].set_title(f"Experiment conversion rate, {priors_label} posterior predictive")

        plt.show()
        plt.close()

        # Plot outcome posterior
        _ = az.plot_posterior(
            self.posterior_predictive.posterior["relative_uplift"],
            kind = kind
            )
        _ = plt.axvline(true_relative_uplift, color = "red")
        _ = plt.annotate(str(round(true_relative_uplift, 2)), (true_relative_uplift, 0.1))
        _ = plt.title(f"Relative uplift, {priors_label} posterior predictive")

        plt.show()
        plt.close()

In [11]:
def generate_variant_data(days: int, conversion_rates: List[float]):

    # Assuming conversion_rates[0] is control
    variant_data_list = []
    
    for rate in conversion_rates:
        visitors = get_daily_visitors(days, lam_visitors, lam_visitors_noise, rng)
        conversions = get_daily_conversions(visitors, rate, conversion_noise_alpha, conversion_noise_beta, rng)
        variant_data = VariantData(sum(visitors), sum(conversions))
        variant_data_list.append(variant_data)
    
    return variant_data_list

## Preposterior analysis (EVSI estimation)

In [12]:
# # Considered practically relevant effect sizes (relative uplift)
# effect_sizes = [
#     0.01,
#     0.1,
#     true_relative_uplift,  # 0.33
#     0.5,
#     0.8
# ]

# Considered number of days
n_days = np.arange(1, 10, 2)

# Considered priors
weak_prior = [
    ConversionPrior(1, 5),
    ConversionPrior(1.5, 5.5)
]

medium_prior = [
    ConversionPrior(15, 85),
    ConversionPrior(25, 100)
]

strong_prior = [
    ConversionPrior(60, 340),
    ConversionPrior(100, 400)
]

priors = {
    "weak": weak_prior, 
    "medium": medium_prior, 
    "strong": strong_prior
}

# Dictionary of parameters
dict_parameters = {
    #"effect_sizes": effect_sizes,
    "n_days": n_days,
    "priors": priors.keys()
}

trials = 50
chains = 4
cores = 4

In [ ]:
# Calculate "loss reduction" for each parameter combination
#effects_list = []
days_list = []
priors_list = []
losses_list = []
posterior_list = []
for j, config in tqdm(
    enumerate(grid_configs(dict_parameters)),
    desc = "configs",
    total = len(priors) * len(n_days)
    ):

    # Get parameters
    ##effect_size = config["effect_sizes"]
    days = config["n_days"]
    prior_label = config["priors"]

    # Calculate experiment conversion rate relative to control, based on effect size
    #exp_conv = (effect_size + 1) * conversion_control

    loss_trials = []
    posterior_trials = []
    for i in tqdm(
        range(trials),
        desc = "trials",
        total = trials
        ):

        # Create model
        model = ConversionRateModel(
            priors = priors[prior_label],
            data = generate_variant_data(
                days = days, 
                conversion_rates = [conversion_control, conversion_experiment]
            ),
            priors_label = prior_label
        )

        # Get prior predictive, mean estimated effect size
        model.get_prior_predictive()
        mu_prior = model.prior_predictive.prior.relative_uplift.mean()

        # Get posterior predictive, mean estimated effect size
        model.get_posterior_predictive(nuts_sampler = "blackjax", chains = chains, cores = cores, progressbar = False)
        mu_posterior = model.posterior_predictive.posterior.relative_uplift.mean()

        # Calculate & record loss reduction
        # Loss from prior estimate - loss from posterior estimate
        loss_reduction = np.abs(true_relative_uplift - mu_prior) - np.abs(true_relative_uplift - mu_posterior)
        loss_trials.append(loss_reduction)

        # Save posterior predictive sample for trial
        posterior_trials.append(
            np.ravel(model.posterior_predictive.posterior.relative_uplift)
        )
    
    # Save parameters, trial average loss, list of posterior predictive samples for each config
    #effects_list.append(true_relative_uplift)
    days_list.append(days)
    priors_list.append(prior_label)
    losses_list.append(np.mean(loss_trials))
    posterior_list.append(posterior_trials)

Exception ignored in: <function _xla_gc_callback at 0x0000027A0606B880>
Traceback (most recent call last):
  File "c:\Users\PC\Documents\WorkLocal\DataScience\GitHub\BayesianModelingExperiments\venv\Lib\site-packages\jax\_src\lib\__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 
Sampling: [conversion_rate_control, conversion_rate_experiment, outcome_control, outcome_experiment]
Sampling: [conversion_rate_control, conversion_rate_experiment, outcome_control, outcome_experiment]
Sampling: [conversion_rate_control, conversion_rate_experiment, outcome_control, outcome_experiment]


1 effect size, 9 parameter configs (3 sample sizes, 3 priors), 100 trials per config: ~19 min per config MINIMUM, ~3hrs with default sampler MINIMUM

1 effect size, 15 parameter configs (5 sample sizes, 3 priors), 50 trials per config: ~3-10 min per config, ~45-150mins with blackjax sampler

GPU sampler with JAX is also available, but only on Linux.

Refactor EVSI as loss reduction with sample size: 

EVSI = loss(prior_estimate) - loss(posterior_estimate) 
= abs(actual_effect - prior_effect) - abs(actual effect - posterior effect)

If the prior estimate is better, it will have lower loss than the posterior -> negative EVSI

If the posterior estimate is better, it will have lower loss than the prior -> positive EVSI

If prior-posterior estimates are identical, their loss will be identical -> zero EVSI

In [ ]:
# Sample size vs. loss plot
_ = sns.lineplot(
    x = days_list,
    y = losses_list,
    hue = priors_list,
    style = priors_list,
    markers = True
)

In [ ]:
# Posterior predictive distributions plot, for each sample size
unique_days = set(days_list)

fig, ax = plt.subplots(len(unique_days), sharex = True)
fig.suptitle("Posterior predictive distributions by n. of days in simulation")
fig.supxlabel(f"Relative uplift, true value: {round(true_relative_uplift, 2)}")
fig.legend(
    handles = [
        mpatches.Patch(color = "blue", label = "Weak prior"),
        mpatches.Patch(color = "orange", label = "Medium prior"),
        mpatches.Patch(color = "green", label = "Strong prior")
    ],
    loc = "right"
)

for n_days in unique_days:

    ax_idx = list(unique_days).index(n_days)

    trials_array = np.reshape(
        np.array(posterior_list)[np.array(days_list) == n_days],
        (len(priors) * trials, 4000)
    )

    colors = ["blue"] * trials + ["orange"] * trials + ["green"] * trials

    for i in range(0, trials_array.shape[0]):
        sns.kdeplot(
            trials_array[i, :],
            color = colors[i],
            ax = ax[ax_idx],
        )
    
    ax[ax_idx].set_ylabel(f"{n_days} days")
    ax[ax_idx].axvline(true_relative_uplift, color = "red")

## Inference

### Weak prior

### Strong prior

In [40]:
# Create model
strong_prior_model = ConversionRateModel(
    priors = strong_prior,
    data = generate_variant_data(
        days = 1, 
        conversion_rates = [conversion_control, conversion_experiment]
    ),
    priors_label = "strong"
)

In [ ]:
# Prior predictive check
strong_prior_model.plot_prior_predictive()

In [ ]:
strong_prior_model.plot_posterior_predictive()